Übung: Secrets
----------------------

Am Beispiel von MySQL soll der Einsatz von Secrets Demonstriert werden.

Zuerst müssen die Secrets mittels `kubectl` erstellt werden:

In [13]:
! kubectl create secret generic mysql-pass --from-literal=password=admin
! kubectl create secret generic mysql-user-pass --from-literal=password=secret

secret/mysql-pass created
secret/mysql-user-pass created


In [14]:
! kubectl get secrets

NAME                  TYPE                                  DATA   AGE
default-token-dtkgr   kubernetes.io/service-account-token   3      8d
mysql-pass            Opaque                                1      1s
mysql-user-pass       Opaque                                1      0s


Anschliessend sind die Umgebungsvariablen in der YAML Datei so zu setzen, dass sie auf die Secret Werte zugreifen

        env:
        - name: MYSQL_ROOT_PASSWORD
          valueFrom:
           secretKeyRef:
            name: mysql-pass
            key: password
            
        - name: MYSQL_PASSWORD
          valueFrom:
           secretKeyRef:
            name: mysql-user-pass
            key: password

Die anderen Werte, wie Username können fix in der YAML Datei gesetzt werden:

        env:
        - name: MYSQL_USER
          value: test
        - name: MYSQL_DATABASE
          value: test 

In [15]:
! kubectl create -f 09-7-Secrets/mysql.yaml

service/mysql created
deployment.apps/mysql created


Zum Testen verwenden wir [adminer](https://www.adminer.org/de/).

Der Server (Hostname) ist `mysql`, Username, Password und Datenbank ergibt sich aus vorheriger Konfiguration.

In [17]:
! kubectl create -f https://raw.githubusercontent.com/mc-b/duk/master/mysql/adminer.yaml
! kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service adminer -o=jsonpath='{ .spec.ports[0].nodePort }')/"


Error from server (AlreadyExists): error when creating "https://raw.githubusercontent.com/mc-b/duk/master/mysql/adminer.yaml": services "adminer" already exists
Error from server (AlreadyExists): error when creating "https://raw.githubusercontent.com/mc-b/duk/master/mysql/adminer.yaml": deployments.apps "adminer" already exists
http://192.168.178.200:31929

Aufräumen

In [18]:
! kubectl delete -f 09-7-Secrets/mysql.yaml
! kubectl delete -f https://raw.githubusercontent.com/mc-b/duk/master/mysql/adminer.yaml
! kubectl delete secret mysql-pass
! kubectl delete secret mysql-user-pass

service "mysql" deleted
deployment.apps "mysql" deleted
service "adminer" deleted
deployment.apps "adminer" deleted
secret "mysql-pass" deleted
secret "mysql-user-pass" deleted
